In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.fft import fft,fftfreq
import time

## The cell below contains all definitions. 

In [ ]:
def simulate(m1, m2, k1, k2, k3, x1 = 0, x2 = 0, F_1 = lambda x: 0, F_2 = lambda x: 0, v1=0, v2=0, steps = 1000, dt=0.01):
    """Calculates the motion of two identical unit masses with a spring between them, given initial positions, etc.
    Parameters:
    x1, x2 (floats): Initial displacements, zero by default
    m1, m2 (floats): Masses of each oscillator
    k1, k2, k3 (float): Spring constants
    F_1, F_2 (lambda functions): monovariable functions determining forces on each mass as functions of time""".
    mass1 = [x1]
    mass2 = [x2]
    vel1 = [v1]
    vel2 = [v2]
    for i in range(steps):
        F1 = -k1*mass1[-1] - k2*(mass1[-1] - mass2[-1]) + F_1(i*dt)
        F2 = -k3*mass2[-1] - k2*(mass2[-1] - mass1[-1]) + F_2(i*dt)
        dv1 = F1*dt/m1
        dv2 = F2*dt/m2
        vel1.append(vel1[-1] + dv1)
        vel2.append(vel2[-1] + dv2)
        mass1.append(mass1[-1] + vel1[-1]*dt)
        mass2.append(mass2[-1] + vel2[-1]*dt)
    return mass1, mass2, vel1, vel2

def find_maxima(array_x, array_y):
    """Given an array ordered by another array, finds and returns the local maxima"""
    array = {array_x[i]:array_y[i] for i in range(len(array_x))}
    outs = []
    for i in range(1,len(array_x)-1):
        if array[array_x[i]] > array[array_x[i-1]] and array[array_x[i]] > array[array_x[i+1]]:
            outs.append((array_x[i], array_y[i]))
            
    return outs

## The two cells below contain an example code. Change any of dt, steps, M, or k.

In [ ]:
dt = 0.01
steps = int(1000000*dt)
M = 1 # Mass ratio
k = 1.219 # I've made this a random number just so that I can be sure that any behaviour of the 
          # oscillator is not a consquence of the ratio of the spring constants being a whole multiple of
          # this value.
x1,x2,v1,v2 = simulate(1, M, k, 1, k,
                                     F_1 = lambda t: np.sin(5.12412*t), F_2 = lambda t: np.sin(t),
#                                      x1=1,x2=1 # Uncomment to set initial conditions. 
                                     steps = steps, dt=dt)
x1 = np.array(x1)
x2 = np.array(x2) 
time = np.linspace(0, steps*dt + dt, steps+1)
fig2, ax2 = plt.subplots(3)
plotarea = 8000 # Change this to change areas of the subplot that you may want to examine
ax2[0].plot(time[plotarea:], x1[plotarea:], "g", label = "Mass 1")
ax2[0].plot(time[plotarea:], x2[plotarea:], "r", label = "Mass 2") # These plots contain the first bits of the graph
ax2[1].plot(time[:len(x1)-plotarea], x1[:len(x1)-plotarea], "g", label = "Mass 1")
ax2[1].plot(time[:len(x1)-plotarea], x2[:len(x2)-plotarea], "r", label = "Mass 2") # These plots contain the second bits of the graph
ax2[2].plot(time, x1, 'g', label = "Mass 1")
ax2[2].plot(time, x2, 'r', label = "Mass 2") # This plot contains the entire graph
# ax2[2].plot(time, -(-(1/(1-k))*np.sin((k**0.5)*time) + (k/(1-k))*np.sin(time)), 'b') # For comparison with theoretical results?

leg = []
for i in range(3):
    leg.append(ax2[i].legend()) # This creates the appropriate legends


In [ ]:
N = len(x1)
T = dt
yf = fft(x1)
yf2 = fft(x2)
xf = fftfreq(N, T)
fig3, ax3 = plt.subplots(2)
ar=200
ax3[0].plot(xf[:N//ar], 2.0/N * np.abs(yf[0:N//ar]), 'g')
ax3[0].plot(xf[:N//ar], 2.0/N * np.abs(yf2[0:N//ar]), 'r')
ax3[0].grid()

ax3[1].plot(xf[0:N//2], 2.0/N * np.abs(yf)[0:N//2], 'g')
ax3[1].plot(xf[0:N//2], 2.0/N * np.abs(yf2)[0:N//2], 'r')
ax3[1].grid()

## The cell below calculates all the maxima on a given plot. This doesn't encapsulate all the details but it should be enough for a rough idea. 
##### To be done: 
- Find a way to represent correlation in Python. Each element contains pairs of maxima and corresponding X-values; they must be correlated
- Cross-correlation doesn't work, it's too computation-intensive and it yields unreasonably high numbers even for poor correlation simply because the majority of the y-values are zero
- Coupled differential equation solver in Python to compare with theoretical results? The equations of motion reduce to a nonhomogeneous linear second-order differential equation, which should be solvable with a combination of eigenvalue solutions and variation of parameters.

In [ ]:
t0 = time.time()
dt = 0.01
steps = int(1000000*dt)
M = 1 # Mass ratio
k_vec = np.linspace(0.2, 1.8, 100)
time = np.linspace(0, steps*dt + dt, steps+1)

OUT = []

for k in k_vec:
    x1,x2,v1,v2 = simulate(1, M, 1, k, 1,
                                     F_1 = lambda t: np.sin(5*t), F_2 = lambda t: np.sin(t),
                                     steps = steps, dt=dt)
    x1, x2 = np.array(x1), np.array(x2)
    N = len(x1)
    yf = fft(x1)
    yf2 = fft(x2)
    xf = fftfreq(N, dt)
    ymax, ymax2 = find_maxima(xf, 2.0/N * np.abs(yf)), find_maxima(xf, 2.0/N *np.abs(yf2))
    OUT.append((ymax, ymax2))
    
t1 = time.time()
print(t1-t0)